In [1]:
from gensim.models import Word2Vec

In [2]:
model = Word2Vec.load('data/imdb-word2vec.w2v')

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# Importing & preprocessing the dataset

train_ds = text_dataset_from_directory('../neuralnets/aclImdb/train')
test_ds = text_dataset_from_directory('../neuralnets/aclImdb/test')

dfTrain = pd.DataFrame(train_ds.unbatch().as_numpy_iterator(), columns=['text', 'label'])
dfTest = pd.DataFrame(test_ds.unbatch().as_numpy_iterator(), columns=['text', 'label'])
_, xts = train_test_split(dfTest, stratify=dfTest['label'], test_size=0.25)

Found 25001 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
len(model.wv)

47046

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
dfTrain['text'] = dfTrain['text'].map(lambda x: x.decode())
xts['text'] = xts['text'].map(lambda x: x.decode())

/home/jaidevd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
tokenizer = Tokenizer(num_words=len(model.wv))
tokenizer.fit_on_texts(dfTrain['text'].tolist())

In [9]:
train_sequences = tokenizer.texts_to_sequences(dfTrain['text'])

In [10]:
test_sequences = tokenizer.texts_to_sequences(xts['text'])

In [11]:
word_index = tokenizer.word_index

In [12]:
word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'he': 26,
 'be': 27,
 'one': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'who': 34,
 'so': 35,
 'from': 36,
 'like': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'about': 41,
 "it's": 42,
 'out': 43,
 'if': 44,
 'has': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'she': 56,
 'even': 57,
 'my': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'story': 62,
 'really': 63,
 'see': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'were': 68,
 'me': 69,
 'well': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'been': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'do': 78,
 'also': 79,
 'into': 80,
 'people': 81,
 'other': 82,
 '

In [13]:
len(word_index)

88582

In [14]:
MAX_SEQUENCE_LENGTH = max(map(len, train_sequences))

In [15]:
MAX_SEQUENCE_LENGTH

2444

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [17]:
train_labels = dfTrain['label'].values
test_labels = xts['label'].values

In [40]:
# Prepare the embedding layer

In [18]:
import numpy as np

In [19]:
embedding_matrix = np.zeros((len(word_index) + 1, model.wv.vector_size))

In [20]:
for word, i in word_index.items():
    try:
        vector = model.wv.get_vector(word, False)
        embedding_matrix[i] = vector
    except KeyError:
        continue

In [21]:
(embedding_matrix.sum(axis=1) == 0).sum()

61067

In [22]:
from tensorflow.keras.layers import Embedding

In [23]:
el = Embedding(len(word_index) + 1, model.wv.vector_size, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)

In [24]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential

In [25]:
clf = Sequential([
    Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'),
    el,
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [31]:
from keras.optimizers import SGD, Adam

In [32]:
clf.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [33]:
clf.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=10, batch_size=128)

Epoch 1/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6926 - accuracy: 0.4985 - val_loss: 0.6921 - val_accuracy: 0.5042
Epoch 2/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6922 - accuracy: 0.5013 - val_loss: 0.6924 - val_accuracy: 0.5038
Epoch 3/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6921 - accuracy: 0.5012 - val_loss: 0.6921 - val_accuracy: 0.5044
Epoch 4/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6921 - accuracy: 0.5051 - val_loss: 0.6922 - val_accuracy: 0.5045
Epoch 5/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6920 - accuracy: 0.5036 - val_loss: 0.6921 - val_accuracy: 0.5044
Epoch 6/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6921 - accuracy: 0.5062 - val_loss: 0.6920 - val_accuracy: 0.5051
Epoch 7/10
196/196 [==============================] - 12s 61ms/step - loss: 0.6920 - accuracy: 0.5038 - val_loss: 0.6920 - val_accuracy:

In [78]:
# With Glove

In [34]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = el(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [36]:
clf.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=100, batch_size=128)

Epoch 1/100
196/196 [==============================] - 12s 61ms/step - loss: 0.6919 - accuracy: 0.5028 - val_loss: 0.6920 - val_accuracy: 0.5051
Epoch 2/100
196/196 [==============================] - 12s 61ms/step - loss: 0.6919 - accuracy: 0.5024 - val_loss: 0.6922 - val_accuracy: 0.5043
Epoch 3/100
196/196 [==============================] - 12s 60ms/step - loss: 0.6919 - accuracy: 0.5052 - val_loss: 0.6920 - val_accuracy: 0.5050
Epoch 4/100
196/196 [==============================] - 12s 60ms/step - loss: 0.6919 - accuracy: 0.5027 - val_loss: 0.6921 - val_accuracy: 0.5051
Epoch 5/100
196/196 [==============================] - 12s 60ms/step - loss: 0.6919 - accuracy: 0.5027 - val_loss: 0.6920 - val_accuracy: 0.5047
Epoch 6/100
196/196 [==============================] - 12s 60ms/step - loss: 0.6919 - accuracy: 0.4967 - val_loss: 0.6920 - val_accuracy: 0.5052
Epoch 7/100
196/196 [==============================] - 12s 60ms/step - loss: 0.6919 - accuracy: 0.5061 - val_loss: 0.6920 - val_ac